In [61]:
"""
https://adventofcode.com/2021/day/3
"""

'\nhttps://adventofcode.com/2021/day/3\n'

# PART 1:

- Aim is to calculate the power consumption via the product of the gamma and epsilon rate.
 - most common bit(value) in the index direction. For each column
 - concatenated  all for gamma rate
 - convert bit to decimal
 
 - epsilon rate the same steps but the least common

In [62]:
RAW = """00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010"""

LINES = [bit for bit in RAW.split("\n")]

from collections import Counter

def get_bit_from_text(text:str):
    LINES = [bit for bit in RAW.split("\n")]
    m = len(LINES)
    n = len(LINES[0])
    most_common = {}

    for i in range(n):
        most_common[i] = list()
        for j in range(m):
              most_common[i].append(LINES[j][i])
        most_common[i] = Counter(most_common[i]).most_common()
    return most_common

def gamma_epsilon(most_common:dict[int:list[tuple[str, int]]]):
    gamma, epsilon = list(), list()
    for i, mc in most_common.items():
        gamma.append(mc[0][0])
        epsilon.append(mc[1][0])
    gamma = int("".join(gamma), 2)
    epsilon = int("".join(epsilon), 2)
    return gamma, epsilon
    

most_common = get_bit_from_text(RAW)
gamma, epsilon = gamma_epsilon(most_common=most_common)

assert gamma * epsilon == 198

# Part 2

In [63]:
# ive decided to used pandas here
# dont judge me :) 
import pandas as pd


# create frame from RAW
df = pd.DataFrame([list(l) for l in LINES])

def subset_number_by_most_common(df:pd.DataFrame, j:int, rating)-> pd.DataFrame:
    temp_df = df.copy()
    counter = temp_df[j].value_counts()
    val_0, val_1 = counter.get('0', 0), counter.get('1', 0)
    if rating == 'O2':
        if val_0 > val_1:
            final_df = temp_df[temp_df[j] == '0']
        else:
            final_df = temp_df[temp_df[j] == '1']
    elif rating == 'CO2':
        if val_0 <= val_1:
            final_df = temp_df[temp_df[j] == '0']
        else:
            final_df = temp_df[temp_df[j] == '1']
    else:
        raise NotImplementedError('Only  CO2 or O2 ratings are implemented')
    return final_df
            

In [64]:
def get_rating(df:pd.DataFrame, rating:str):
    
    temp_df = df.copy()
    _, J = temp_df.shape
    for j in range(J):
        temp_df = subset_number_by_most_common(df=temp_df,
                                               j=j,
                                               rating=rating)
        if len(temp_df) == 1:
            return int("".join(temp_df.values[0]), 2)

In [65]:
with open('inputs/day3.txt') as f:
    PUZZLE = f.read()
    most_common = get_bit_from_text(PUZZLE)
    gamma, epsilon = gamma_epsilon(most_common=most_common)
    power = gamma * epsilon
    print("p1", power)
    df = pd.DataFrame([list(line)  
                        for line in PUZZLE.split("\n")
                        if line])
    o2, co2  = get_rating(df,'O2'), get_rating(df,'CO2')
    life_support_rat = o2 * co2
    print("p2", life_support_rat)

p1 198
p2 3414905


In [66]:
# JOEL G solution
# Consulted after
## PART 1

# cool way to use counter accross
# ive used dict 
INPUT = RAW.splitlines()
def gamma_rate(numbers: list[str]) -> str:
    m = len(numbers[0])
    counts = [
        Counter(number[i] for number in numbers)
        for i in range(m)
    ]

    return ''.join(
        counter.most_common(1)[0][0]
        for counter in counts
    )

# really cool way to flip 
# gamma to epsilon rate
def epsilon_rate(numbers: list[str]) -> str:
    gr = gamma_rate(numbers)
    return ''.join('1' if c == '0' else '0' for c in gr)

assert gamma_rate(INPUT) == "10110"
assert epsilon_rate(INPUT) == "01001"

# PART 2

# NOT using pandas
# simple way not using pandas
def oxygen_generator_rating(numbers: list[str]) -> str:
    m = len(numbers[0])

    for i in range(m):
        zeros = 0
        ones = 0
        for number in numbers:
            if number[i] == '0':
                zeros += 1
            elif number[i] == '1':
                ones += 1

        if ones >= zeros:
            numbers = [n for n in numbers if n[i] == '1']
        else:
            numbers = [n for n in numbers if n[i] == '0']

        if len(numbers) == 1:
            return numbers[0]

    raise ValueError("No solution found")

assert oxygen_generator_rating(INPUT) == "10111"